In [101]:
# tratando os dados

import pandas as pd

df_raw = pd.read_csv("data/abt_churn.csv")

#defining the df we are using for the model
filtro = df_raw["dtRef"] != '2025-04-01'
filtro_1 = df_raw["dtRef"] == '2025-04-01'

df_train = df_raw[filtro].copy().reset_index(drop=True)
oot = df_raw[filtro_1].copy().reset_index(drop=True) # out of time, a ultima leva de dados, o prof sugeriu n usar 

df_train.head()
# df_train.shape, oot.shape

,dtRef,idUsuario,qtdeTransacoes,qtdeDias,mediaTransacoesDias,saldoPontos,qtdePontosPos,qtdePontosNeg,qtdeDiasUltimaTransacao,qtdeDiasPrimeiraTransacao,...,saldoPontosD28,qtdePontosPosD28,qtdePontosNegD28,propAvgQtdeTransacoes,propAvgQtdeDias,propAvgMediaTransacoesDias,propAvgSaldoPontos,propAvgQtdePontosPos,propAvgQtdePontosNeg,flagChurn
0,2024-06-01,000ff655-fa9f-4baa-a108-47f581ec52a1,266,27,9.851852,635,2635,-2000,1.0,89.0,...,151,151,0,3.889781,3.278281,2.135602,1.516314,4.101926,8.944444,1
1,2024-10-01,000ff655-fa9f-4baa-a108-47f581ec52a1,268,28,9.571429,686,2686,-2000,4.0,211.0,...,51,51,0,3.309865,2.567615,2.328737,1.288278,2.920954,5.167037,1
2,2024-04-01,000ff655-fa9f-4baa-a108-47f581ec52a1,188,11,17.090909,275,1275,-1000,3.0,28.0,...,275,1275,-1000,3.805468,1.820201,3.739048,0.798622,2.634551,7.162791,0
3,2024-05-01,000ff655-fa9f-4baa-a108-47f581ec52a1,262,24,10.916667,484,2484,-2000,2.0,58.0,...,107,1107,-1000,4.246216,3.255244,2.322640,1.275749,4.275765,9.922414,0
4,2024-08-01,001749bd-37b5-4b1e-8111-f9fbba90f530,1,1,1.000000,50,50,0,21.0,21.0,...,50,50,0,0.013317,0.103973,0.236635,0.104533,0.063302,0.000000,1


In [102]:
# checando se há usuarios duplicados nos dados

check_duplicados = df_train.groupby("idUsuario").agg({"idUsuario": "count"}).copy()

check_duplicados

num_users_more_than_one_entry = (check_duplicados['idUsuario'] > 1).sum()
print(num_users_more_than_one_entry)

1048


In [103]:
# definiçoes do modelo

features = df_train.drop(columns=["idUsuario", "dtRef", "flagChurn"]).columns.tolist()

X, y = df_train[features], df_train["flagChurn"]

In [104]:
# Sampling

from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.2, 
                                                                    random_state=42,
                                                                    stratify=y
                                                                    )

print(y.mean(), " , ", y_train.mean(), " , ", y_test.mean())

0.4689004429039091  ,  0.46894559460760715  ,  0.4687199230028874


In [105]:
# Exploring [EDA - Explore Data Analysis]

X_train.isna().sum() # nao tem nenhum dado faltante [MISSING]

df_analyse = X_train.copy()
df_analyse["target"] = y_train.copy()

# exploratory analysis of the features/means

summary = df_analyse.groupby("target").agg(["mean", "median"]).copy().T
summary_mean = summary[summary.index.get_level_values(1) == "mean"].copy()

summary_mean["dif_relat"] = ((summary_mean[0] / summary_mean[1]) / (summary_mean[0] / summary_mean[1]).sum())
summary_mean = summary_mean.sort_values(by="dif_relat", ascending=False)
summary_mean["dif_relat_aculm"] = summary_mean["dif_relat"].cumsum()

filtro_summary = (summary_mean["dif_relat_aculm"] <= 0.95) & (summary_mean["dif_relat"] > 0.01)

summary_mean = summary_mean[filtro_summary].copy()
summary_mean

,target,0,1,dif_relat,dif_relat_aculm
qtdePontosNegD14,mean,-136.903445,-7.007187,0.077613,0.077613
qtdePontosNegD7,mean,-67.429737,-3.644764,0.073493,0.151106
qtdePontosNegD28,mean,-300.072076,-21.765914,0.054766,0.205872
qtdeRLover,mean,0.067090,0.006160,0.043264,0.249135
qtdeChurn,mean,5.716682,0.622177,0.036500,0.285635
qtdePontosPosD7,mean,159.402539,18.478953,0.034267,0.319903
qtdeAirflowLover,mean,0.126020,0.014887,0.033627,0.353530
qtdePontosPosD14,mean,316.513599,38.107803,0.032994,0.386525
qtdeTrocaStreamElements,mean,1.480054,0.180185,0.032630,0.419155
propAvgQtdePontosNeg,mean,4.993870,0.612794,0.032373,0.451528


In [113]:
# checking the features I analysed last but with a decision tree

from sklearn import tree
import matplotlib.pyplot as plt

tree_model = tree.DecisionTreeClassifier(max_depth=5, random_state=42)
tree_model.fit(X_train,y_train)

plt.figure(figsize=(40,32), dpi=300)
tree.plot_tree(tree_model,
               filled=True,
               feature_names=X_train.columns)
plt.show()

In [1]:
from sklearn import linear_model
from sklearn import naive_bayes